#Move to Folder

In [ ]:
!ls

bidafc2id.pickle  bidafvalid.pkl    evaluate.py  preprocess.py
bidafglove.npy	  bidafw2id.pickle  GloVe	 __pycache__
bidaftrain.pkl	  data		    main.py


In [17]:
%cd drive/MyDrive/Project/squad/bidaf

[Errno 2] No such file or directory: 'drive/MyDrive/Project/squad/bidaf'
/content/drive/MyDrive/Project/squad


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter

from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
nlp = spacy.blank('en')
# from preprocess import *
%load_ext autoreload
%autoreload 2

#Preprocessing

In [ ]:
def load_json(path):
    '''
    Loads the JSON file of the Squad dataset.
    Returns the json object of the dataset.
    '''
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    print("Length of data: ", len(data['data']))
    print("Data Keys: ", data['data'][0].keys())
    print("Title: ", data['data'][0]['title'])
    
    return data

In [ ]:
def parse_data(data:dict)->list:
    '''
    Parses the JSON file of Squad dataset by looping through the
    keys and values and returns a list of dictionaries with
    context, query and label triplets being the keys of each dict.
    '''
    data = data['data']
    qa_list = []
    gk = []
    dk = []

    for paragraphs in data:

        for para in paragraphs['paragraphs']:
            context = para['context']
            if "§" in context.lower():
                if context not in gk:
                    print(context)
                    gk.append(context)
                continue

            for qa in para['qas']:
                
                id = qa['id']
                question = qa['question']
                if "§" in question.lower():
                    if question not in dk:
                        print("q")
                        print(question)
                        dk.append(question)
                    continue
                
                
                for ans in qa['answers']:
                    answer = ans['text']
                    ans_start = ans['answer_start']
                    ans_end = ans_start + len(answer)
                    
                    qa_dict = {}
                    qa_dict['id'] = id
                    qa_dict['context'] = context
                    qa_dict['question'] = question
                    qa_dict['label'] = [ans_start, ans_end]

                    qa_dict['answer'] = answer
                    qa_list.append(qa_dict)    

    
    return qa_list

In [ ]:
def gather_text_for_vocab(dfs:list):
    '''
    Gathers text from contexts and questions to build a vocabulary.
    
    :param dfs: list of dataframes of SQUAD dataset.
    :returns: list of contexts and questions
    '''
    
    text = []
    total = 0
    for df in dfs:
        unique_contexts = list(df.context.unique())
        unique_questions = list(df.question.unique())
        total += df.context.nunique() + df.question.nunique()
        text.extend(unique_contexts + unique_questions)
    
    assert len(text) == total
    
    return text

In [ ]:
def build_word_vocab(vocab_text):
    '''
    Builds a word-level vocabulary from the given text.
    
    :param list vocab_text: list of contexts and questions
    :returns 
        dict word2idx: word to index mapping of words
        dict idx2word: integer to word mapping
        list word_vocab: list of words sorted by frequency
    '''
    
    
    words = []
    for sent in vocab_text:
        for word in nlp(sent, disable=['parser','tagger','ner']):
            words.append(word.text)

    word_counter = Counter(words)
    word_vocab = sorted(word_counter, key=word_counter.get, reverse=True)
    print(f"raw-vocab: {len(word_vocab)}")
    #word_vocab = list(set(word_vocab).intersection(set(glove_words)))
    print(f"glove-vocab: {len(word_vocab)}")
    word_vocab.insert(0, '')
    word_vocab.insert(1, '')
    print(f"vocab-length: {len(word_vocab)}")
    word2idx = {word:idx for idx, word in enumerate(word_vocab)}
    print(f"word2idx-length: {len(word2idx)}")
    idx2word = {v:k for k,v in word2idx.items()}
    
    
    return word2idx, idx2word, word_vocab


In [ ]:
def build_char_vocab(vocab_text):
    '''
    Builds a character-level vocabulary from the given text.
    
    :param list vocab_text: list of contexts and questions
    :returns 
        dict char2idx: character to index mapping of words
        list char_vocab: list of characters sorted by frequency
    '''
    
    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'')
    char_vocab.insert(1,'')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

In [ ]:
def context_to_ids(text, word2idx):
    '''
    Converts context text to their respective ids by mapping each word
    using word2idx. Input text is tokenized using spacy tokenizer first.
    
    :param str text: context text to be converted
    :returns list context_ids: list of mapped ids
    
    :raises assertion error: sanity check
    
    '''

    context_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids
    
def question_to_ids(text, word2idx):
    '''
    Converts question text to their respective ids by mapping each word
    using word2idx. Input text is tokenized using spacy tokenizer first.
    
    :param str text: question text to be converted
    :returns list context_ids: list of mapped ids
    
    :raises assertion error: sanity check
    
    '''

    question_tokens = [w.text for w in nlp(text, disable=['parser','tagger','ner'])]
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

In [ ]:
pg = "n 1992 he started working at the Eindhoven University of Technology as an assistant professor for the department of Mathematics and Computing Science, where he headed the Specification and Modeling of Information systems (SMIS) research group. 2000 to 2003 he became a part-time full professor at the Computing Science department. And from 2000 to 2006 he was head of the Information Systems department at the Technology Management department of TU/e. Since 2006 he is full professor at the Department of Mathematics & Computer Science of the Eindhoven University of Technology. He also has a part-time appointment in the BPM group of Queensland University of Technology (QUT)"

In [ ]:
s, e = zip(*[(word.idx, word.idx+len(word.text)) for word in nlp(pg)])

In [ ]:
def test_indices(df, idx2word):
    '''
    Performs the tests mentioned above. This method also gets the start and end of the answers
    with respect to the context_ids for each example.
    
    :param dataframe df: SQUAD df
    :returns
        list start_value_error: example idx where the start idx is not found in the start spans
                                of the text
        list end_value_error: example idx where the end idx is not found in the end spans
                              of the text
        list assert_error: examples that fail assertion errors. A majority are due to the above errors
        
    '''

    start_value_error = []
    end_value_error = []
    assert_error = []
    for index, row in df.iterrows():

        answer_tokens = [w.text for w in nlp(row['answer'], disable=['parser','tagger','ner'])]

        start_token = answer_tokens[0]
        end_token = answer_tokens[-1]
        
        context_span  = [(word.idx, word.idx + len(word.text)) 
                         for word in nlp(row['context'], disable=['parser','tagger','ner'])]

        starts, ends = zip(*context_span)

        answer_start, answer_end = row['label']

        try:
            start_idx = starts.index(answer_start)
        except:
            start_value_error.append(index)
        try:
            end_idx  = ends.index(answer_end)
        except:
            end_value_error.append(index)

        try:
            assert idx2word[row['context_ids'][start_idx]] == answer_tokens[0]
            assert idx2word[row['context_ids'][end_idx]] == answer_tokens[-1]
        except:
            assert_error.append(index)


    return start_value_error, end_value_error, assert_error

In [ ]:
def get_error_indices(df, idx2word):
    '''
    Gets error indices from the method above and returns a 
    set of those indices.
    '''
    
    start_value_error, end_value_error, assert_error = test_indices(df, idx2word)
    err_idx = start_value_error + end_value_error + assert_error
    err_idx = set(err_idx)
    print(f"Error indices: {len(err_idx)}")
    
    return err_idx

In [ ]:
def index_answer(row, idx2word):
    '''
    Takes in a row of the dataframe or one training example and
    returns a tuple of start and end positions of answer by calculating 
    spans.
    '''
    
    context_span = [(word.idx, word.idx + len(word.text)) for word in nlp(row.context, disable=['parser','tagger','ner'])]
    starts, ends = zip(*context_span)
    
    answer_start, answer_end = row.label
    start_idx = starts.index(answer_start)
 
    end_idx  = ends.index(answer_end)
    
    ans_toks = [w.text for w in nlp(row.answer,disable=['parser','tagger','ner'])]
    ans_start = ans_toks[0]
    ans_end = ans_toks[-1]
    assert idx2word[row.context_ids[start_idx]] == ans_start
    assert idx2word[row.context_ids[end_idx]] == ans_end
    
    return [start_idx, end_idx]

#Preprocess Data

In [ ]:
train_data = load_json('./data/squad_train.json')
valid_data = load_json('./data/squad_dev.json')

# parse the json structure to return the data as a list of dictionaries

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)
print('--------------------------')

print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))

# converting the lists into dataframes

train_df = pd.DataFrame(train_list)
valid_df = pd.DataFrame(valid_list)

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50
Congress often writes legislation to restrain executive officials to the performance of their duties, as laid out by the laws Congress passes. In INS v. Chadha (1983), the Supreme Court decided (a) The prescription for legislative action in Art. I, § 1—requiring all legislative powers to be vested in a Congress consisting of a Senate and a House of Representatives—and § 7—requiring every bill passed by the House and Senate, before becoming law, to be presented to the president, and, if he disapproves, to be repassed by two-thirds of the Senate and House—represents the Framers' decision that the legislative power of the Federal Government be exercised in accord with a single, finely wrought and exhaustively considered procedure. This procedure is an integral part of the constitutional design for the 

In [ ]:
train_df.head()

,id,context,question,label,answer
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"[515, 541]",Saint Bernadette Soubirous
1,5733be284776f4190066117f,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"[188, 213]",a copper statue of Christ
2,5733be284776f41900661180,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"[279, 296]",the Main Building
3,5733be284776f41900661181,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,"[381, 420]",a Marian place of prayer and reflection
4,5733be284776f4190066117e,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,"[92, 126]",a golden statue of the Virgin Mary


In [ ]:
def preprocess_df(df):
    
    def to_lower(text):
        return text.lower()

    df.context = df.context.apply(to_lower)
    df.question = df.question.apply(to_lower)
    df.answer = df.answer.apply(to_lower)

In [ ]:
preprocess_df(train_df)
preprocess_df(valid_df)

train_df.head()

,id,context,question,label,answer
0,5733be284776f41900661182,"architecturally, the school has a catholic cha...",to whom did the virgin mary allegedly appear i...,"[515, 541]",saint bernadette soubirous
1,5733be284776f4190066117f,"architecturally, the school has a catholic cha...",what is in front of the notre dame main building?,"[188, 213]",a copper statue of christ
2,5733be284776f41900661180,"architecturally, the school has a catholic cha...",the basilica of the sacred heart at notre dame...,"[279, 296]",the main building
3,5733be284776f41900661181,"architecturally, the school has a catholic cha...",what is the grotto at notre dame?,"[381, 420]",a marian place of prayer and reflection
4,5733be284776f4190066117e,"architecturally, the school has a catholic cha...",what sits on top of the main building at notre...,"[92, 126]",a golden statue of the virgin mary


In [ ]:
vocab_text = gather_text_for_vocab([train_df, valid_df])
print("Number of sentences in dataset: ", len(vocab_text))

Number of sentences in dataset:  118745


In [ ]:
vocab_text[111130]

"what political party is strongest in melbourne's working class suburbs?"

In [ ]:
word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)
print("----------------------------------")
char2idx, char_vocab = build_char_vocab(vocab_text)

In [ ]:
train_df['context_ids'] = train_df.context.apply(context_to_ids, word2idx=word2idx)
valid_df['context_ids'] = valid_df.context.apply(context_to_ids, word2idx=word2idx)
train_df['question_ids'] = train_df.question.apply(question_to_ids, word2idx=word2idx)
valid_df['question_ids'] = valid_df.question.apply(question_to_ids, word2idx=word2idx)

In [ ]:
train_df.head()

,id,context,question,label,answer,context_ids,question_ids
0,5733be284776f41900661182,"architecturally, the school has a catholic cha...",to whom did the virgin mary allegedly appear i...,"[515, 541]",saint bernadette soubirous,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[9, 569, 25, 2, 2679, 851, 5993, 1082, 6, 8054..."
1,5733be284776f4190066117f,"architecturally, the school has a catholic cha...",what is in front of the notre dame main building?,"[188, 213]",a copper statue of christ,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 12, 6, 1202, 4, 2, 1259, 1196, 237, 300, 7]"
2,5733be284776f41900661180,"architecturally, the school has a catholic cha...",the basilica of the sacred heart at notre dame...,"[279, 296]",the main building,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[2, 4546, 4, 2, 3913, 1498, 31, 1259, 1196, 12..."
3,5733be284776f41900661181,"architecturally, the school has a catholic cha...",what is the grotto at notre dame?,"[381, 420]",a marian place of prayer and reflection,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 12, 2, 19586, 31, 1259, 1196, 7]"
4,5733be284776f4190066117e,"architecturally, the school has a catholic cha...",what sits on top of the main building at notre...,"[92, 126]",a golden statue of the virgin mary,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 8834, 24, 402, 4, 2, 237, 300, 31, 1259, ..."


In [ ]:
train_err = get_error_indices(train_df, idx2word)
valid_err = get_error_indices(valid_df, idx2word)

Error indices: 920
Error indices: 349


In [ ]:
train_df.drop(train_err, inplace=True)
valid_df.drop(valid_err, inplace=True)

In [ ]:
train_label_idx = train_df.apply(index_answer, axis=1, idx2word=idx2word)
valid_label_idx = valid_df.apply(index_answer, axis=1, idx2word=idx2word)

train_df['label_idx'] = train_label_idx
valid_df['label_idx'] = valid_label_idx


In [ ]:
train_df.head()

,id,context,question,label,answer,context_ids,question_ids,label_idx
0,5733be284776f41900661182,"architecturally, the school has a catholic cha...",to whom did the virgin mary allegedly appear i...,"[515, 541]",saint bernadette soubirous,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[9, 569, 25, 2, 2679, 851, 5993, 1082, 6, 8054...","[102, 104]"
1,5733be284776f4190066117f,"architecturally, the school has a catholic cha...",what is in front of the notre dame main building?,"[188, 213]",a copper statue of christ,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 12, 6, 1202, 4, 2, 1259, 1196, 237, 300, 7]","[37, 41]"
2,5733be284776f41900661180,"architecturally, the school has a catholic cha...",the basilica of the sacred heart at notre dame...,"[279, 296]",the main building,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[2, 4546, 4, 2, 3913, 1498, 31, 1259, 1196, 12...","[57, 59]"
3,5733be284776f41900661181,"architecturally, the school has a catholic cha...",what is the grotto at notre dame?,"[381, 420]",a marian place of prayer and reflection,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 12, 2, 19586, 31, 1259, 1196, 7]","[76, 82]"
4,5733be284776f4190066117e,"architecturally, the school has a catholic cha...",what sits on top of the main building at notre...,"[92, 126]",a golden statue of the virgin mary,"[16255, 3, 2, 133, 40, 10, 544, 793, 5, 8718, ...","[11, 8834, 24, 402, 4, 2, 237, 300, 31, 1259, ...","[17, 23]"


In [ ]:
train_df["answer"][0]

'saint bernadette soubirous'

In [ ]:
train_df["context"][0][515:541]

'saint bernadette soubirous'

In [ ]:
nlp(train_df["context"][0])[102:105]

saint bernadette soubirous

In [ ]:
train_df.to_pickle('bidaftrain.pkl')
valid_df.to_pickle('bidafvalid.pkl')

with open('bidafw2id.pickle','wb') as handle:
    pickle.dump(word2idx, handle)

with open('bidafc2id.pickle','wb') as handle:
    pickle.dump(char2idx, handle)

#Data Loader

In [19]:
train_df = pd.read_pickle('bidaftrain.pkl')
valid_df = pd.read_pickle('bidafvalid.pkl')



In [20]:
with open('bidafw2id.pickle','rb') as handle:
    word2idx = pickle.load(handle)
with open('bidafc2id.pickle','rb') as handle:
    char2idx = pickle.load(handle)

idx2word = {v:k for k,v in word2idx.items()}

In [21]:
import torch.utils.data as data

class SQuAD:
    def __init__(self, data, batch_size):
        # super(SQuAD, self).__init__()
        
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data

    def __len__(self):
        return len(self.data)

    def make_char_vector(self, max_sent_len, max_word_len, sentence):
        
        char_vec = torch.ones(max_sent_len, max_word_len).type(torch.LongTensor)
        
        for i, word in enumerate(nlp(sentence, disable=['parser','tagger','ner'])):
            for j, ch in enumerate(word.text):
                char_vec[i][j] = char2idx.get(ch, 0)
        
        return char_vec 

    def get_span(self, text):
        
        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span

    def __iter__(self):
        for batch in self.data:
            
            spans = []
            ctx_text = []
            answer_text = []
            
            for ctx in batch.context:
                ctx_text.append(ctx)
                spans.append(self.get_span(ctx))
            
            for ans in batch.answer:
                answer_text.append(ans)
                
            
            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)
            
            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)
                
            max_word_ctx = 0
            for context in batch.context:
                for word in nlp(context, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ctx:
                        max_word_ctx = len(word.text)
            
            char_ctx = torch.ones(len(batch), max_context_len, max_word_ctx).type(torch.LongTensor)
            for i, context in enumerate(batch.context):
                char_ctx[i] = self.make_char_vector(max_context_len, max_word_ctx, context)
            
            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)
            
            for i, ques in enumerate(batch.question_ids):
                padded_question[i, :len(ques)] = torch.LongTensor(ques)
                
            max_word_ques = 0
            for question in batch.question:
                for word in nlp(question, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ques:
                        max_word_ques = len(word.text)
            
            char_ques = torch.ones(len(batch), max_question_len, max_word_ques).type(torch.LongTensor)
            for i, question in enumerate(batch.question):
                char_ques[i] = self.make_char_vector(max_question_len, max_word_ques, question)
            
            ids = list(batch.id)  
            label = torch.LongTensor(list(batch.label_idx))
            
            yield (padded_context, padded_question, char_ctx, char_ques, label, ctx_text, answer_text, ids)


        

In [22]:
train_dataset = SQuAD(train_df[:50], 8)
valid_dataset = SQuAD(valid_df[-38:-12], 8)

In [23]:
a = next(iter(train_dataset))

#BiDAF

##Word Level Embedding

In [ ]:
def get_glove_dict():
    glove_dict = {}
    with open("GloVe/glove.840B.300d.txt", "r", encoding="utf-8") as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            glove_dict[word] = vector
            
    f.close()
    
    return glove_dict

In [ ]:
glove_dict = get_glove_dict()

In [ ]:
glove_dict["a"]

array([ 4.3798e-02,  2.4779e-02, -2.0937e-01,  4.9745e-01,  3.6019e-01,
       -3.7503e-01, -5.2078e-02, -6.0555e-01,  3.6744e-02,  2.2085e+00,
       -2.3389e-01, -6.8360e-02, -2.2355e-01, -5.3989e-02, -1.5198e-01,
       -1.7319e-01,  5.3355e-02,  1.6485e+00, -4.7991e-02, -8.5311e-02,
       -1.5712e-01, -6.4425e-01, -3.9819e-01,  2.7800e-01,  1.5364e-01,
        3.1678e-02,  5.5414e-02,  1.5939e-02,  3.1851e-01, -5.8979e-02,
        3.8584e-02,  1.0770e-01,  1.0410e-01, -7.7346e-02,  3.7396e-01,
       -2.1482e-01,  3.8320e-01, -2.7737e-01, -1.8352e-01, -8.3838e-01,
        3.4124e-01,  5.8164e-01,  1.8543e-01, -3.1028e-01,  1.7666e-01,
       -6.9421e-02, -3.4422e-01, -1.3665e-01, -1.0823e-01,  2.3637e-01,
       -3.2923e-01,  6.1348e-01,  1.9720e-01,  8.7123e-02,  1.0785e-01,
        3.0730e-01,  1.3757e-01,  3.0809e-01,  2.4331e-01, -2.9422e-01,
       -9.8214e-03,  5.5675e-01, -4.8880e-02,  9.9468e-02,  3.0543e-01,
       -3.7597e-01, -1.9525e-01,  4.6246e-02, -3.6675e-02,  3.40

In [ ]:
word_vocab = list(word2idx.keys())

In [ ]:
def create_weights_matrix(glove_dict):
    weights_matrix = np.zeros((len(word_vocab), 300))
    words_found = 0
    for i, word in enumerate(word_vocab):
        try:
            weights_matrix[i] = glove_dict[word]
            words_found += 1
        except:
            pass
        
    return weights_matrix, words_found

In [ ]:
weights_matrix, words_found = create_weights_matrix(glove_dict)
print("Words found in the GloVe vocab: " ,words_found)

Words found in the GloVe vocab:  67397


In [ ]:
len(word_vocab)

96775

In [ ]:
weights_matrix.shape

(96775, 100)

In [ ]:
np.save('bidafglove.npy', weights_matrix)

##Character Level Embedding

In [24]:
class CharacterEmbeddingLayer(nn.Module):
    def __init__(self, char_vocab_dim, char_emb_dim, num_output_channels, kernel_size, drop_prob=0.2):
        super(CharacterEmbeddingLayer, self).__init__()
        self.char_emb_dim = char_emb_dim
        self.char_embedding = nn.Embedding(char_vocab_dim, char_emb_dim, padding_idx=1)
        self.char_convolution = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.dropout(self.char_embedding(x))
        x = x.permute(0, 1, 3, 2)
        x = x.view(-1, self.char_emb_dim, x.shape[3])
        x = x.unsqueeze(1)

        x = self.char_convolution(x)
        x = self.relu(x)
        x = x.squeeze()
        if len(list(x.size())) < 3:
            x = x.unsqueeze(2)
        x = F.max_pool1d(x, x.shape[2]).squeeze()

        x = x.view(batch_size, -1, x.shape[-1])

        return(x)

##HighWay Networking

In [25]:
class HighwayNetwork(nn.Module):
    
    def __init__(self, input_dim, num_layers=2):
        
        super(HighwayNetwork, self).__init__()
        
        self.num_layers = num_layers
        
        self.transforms = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.gates = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        
    def forward(self, x):
        for gate, transform in zip(self.gates, self.transforms):
            # Shapes of g, t, and x are all (batch_size, seq_len, hidden_size)
            
            g = gate(x)
            g = torch.sigmoid(g)
            
            t = F.relu(transform(x))
            
            x = g * t + (1 - g) * x

        return x

##Contextual Embeddings

In [40]:
class ContextualEmbeddingLayer(nn.Module):
    
    def __init__(self, input_dim, hidden_dim):
        
        super(ContextualEmbeddingLayer, self).__init__()
        
        self.trans = nn.TransformerEncoderLayer(input_dim, batch_first=True, nhead=8)
        self.linear = nn.Sequential(nn.Linear(input_dim, 2*hidden_dim), nn.ReLU())
        
        self.highway_net = HighwayNetwork(input_dim)
        
    def forward(self, x):
        
        highway_out = self.highway_net(x)
        # highway_out = [bs, seq_len, input_dim]
        
        outputs = self.trans(highway_out)
        outputs = self.linear(outputs)

        # outputs = [bs, seq_len, hidden_dim*2]
        
        return outputs


##BiDAF Model

In [41]:
class BiDAF(nn.Module):
    def __init__(self, char_vocab_dim, emb_dim, char_emb_dim, num_output_channels, 
                 kernel_size, device):
    
        super(BiDAF, self).__init__()
        self.device = device

        self.word_embedding = self.get_glove_embedding()
        self.character_embedding = CharacterEmbeddingLayer(char_vocab_dim, char_emb_dim, num_output_channels, kernel_size)
        self.contextual_embedding = ContextualEmbeddingLayer(2*emb_dim, 2*emb_dim)

        self.dropout = nn.Dropout()
        self.similarity_weight = nn.Linear(6*(2*emb_dim), 1, bias=False)
        self.modeling_trans = nn.TransformerEncoderLayer(8*(2*emb_dim), batch_first=True, nhead=8)
        self.linear1 = nn.Sequential(nn.Linear(8*(2*emb_dim), 2*2*emb_dim), nn.ReLU())
        self.output_start = nn.Linear(10*(2*emb_dim), 1, bias=False)
        self.output_end = nn.Linear(10*(2*emb_dim), 1, bias=False)
        self.end_trans = nn.TransformerEncoderLayer(2*emb_dim*2, batch_first=True, nhead=8)
        self.linear2 = nn.Sequential(nn.Linear(2*emb_dim*2, 2*2*emb_dim), nn.ReLU())

    def get_glove_embedding(self):
        
        weights_matrix = np.load('bidafglove100.npy')
        num_embeddings, embedding_dim = weights_matrix.shape
        embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix).to(self.device),freeze=True)

        return embedding

    def forward(self, ctx, ques, char_ctx, char_ques):
        
        ctx_len = ctx.shape[1]
        ques_len = ques.shape[1]
        
        ctx_word_embed = self.word_embedding(ctx)
        ctx_char_embed = self.character_embedding(char_ctx)
        # print("che")
        ctx_contextual_in = torch.cat([ctx_word_embed, ctx_char_embed], dim=2)
        ctx_contextual_embed = self.contextual_embedding(ctx_contextual_in)
        # print("coe")

        ques_word_embed = self.word_embedding(ques)
        ques_char_embed = self.character_embedding(char_ques)
        # print("qhe")
        ques_contextual_in = torch.cat([ques_word_embed, ques_char_embed], dim=2)
        ques_contextual_embed = self.contextual_embedding(ques_contextual_in)
        # print("qoe")
        # Similarity Matrix
        c = ctx_contextual_embed.unsqueeze(2).repeat(1, 1, ques_len, 1)
        q = ques_contextual_embed.unsqueeze(1).repeat(1, ctx_len, 1, 1)

        elementwise = torch.mul(c, q)
        alpha = torch.cat([c, q, elementwise], dim=3)
        similarity_matrix = self.similarity_weight(alpha).view(-1, ctx_len, ques_len)

        # Context2Query Attention
        a = F.softmax(similarity_matrix, dim=-1)
        c2q = torch.bmm(a, ques_contextual_embed)

        # Query2Context Attention
        b = F.softmax(torch.max(similarity_matrix, 2)[0], dim=-1)
        b = b.unsqueeze(1)
        q2c = torch.bmm(b, ctx_contextual_embed)
        q2c = q2c.repeat(1, ctx_len, 1)

        # Query Aware Representation
        G = torch.cat([ctx_contextual_embed, c2q, torch.mul(ctx_contextual_embed, c2q), torch.mul(ctx_contextual_embed, q2c)], dim=2)

        # Modeling Layer
        M = self.modeling_trans(G)
        M = self.linear1(M)

        # Output Layer
        M2 = self.end_trans(M)
        self.linear2(M)

        p1 = self.output_start(torch.cat([G, M], dim=2)).squeeze()

        p2 = self.output_start(torch.cat([G, M2], dim=2)).squeeze()

        return p1, p2





#Training

In [42]:
CHAR_VOCAB_DIM = len(char2idx)
EMB_DIM = 100
CHAR_EMB_DIM = 8
NUM_OUTPUT_CHANNELS = 100
KERNEL_SIZE = (8,5)
HIDDEN_DIM = 300
device = torch.device('cuda:0')
device = torch.device('cpu')

model = BiDAF(CHAR_VOCAB_DIM, 
              EMB_DIM, 
              CHAR_EMB_DIM, 
              NUM_OUTPUT_CHANNELS, 
              KERNEL_SIZE, 
              device).to(device)


In [43]:
from tqdm import tqdm

In [44]:
import torch.optim as optim
from torch.autograd import Variable
optimizer = optim.Adadelta(model.parameters())

In [45]:
def train(model, train_dataset):
    print("training")
    train_loss = 0.
    batch_count = 0
    model.train()

    for batch in tqdm(train_dataset):
        optimizer.zero_grad()

        if batch_count%500 == 0:
            print(f"Starting batch: {batch_count}")

        batch_count += 1

        context, question, char_ctx, char_ques, label, ctx_text, ans, ids = batch
        context, question, char_ctx, char_ques, label = context.to(device), question.to(device), char_ctx.to(device), char_ques.to(device), label.to(device)

        preds = model(context, question, char_ctx, char_ques)

        start_pred, end_pred = preds
        s_idx, e_idx = label[:,0], label[:,1]

        loss = F.cross_entropy(start_pred, s_idx) + F.cross_entropy(end_pred, e_idx)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()

    return train_loss/len(train_dataset)


In [46]:
def valid(model, valid_dataset):
    print("Validating....")
    valid_loss = 0.
    batch_count = 0
    f1, em = 0., 0.
    model.eval()
    predictions = {}

    for batch in valid_dataset:
        if batch_count % 500 == 0:
            print(f"Starting batch {batch_count}")
        batch_count += 1

        context, question, char_ctx, char_ques, label, ctx, answers, ids = batch
        context, question, char_ctx, char_ques, label = context.to(device), question.to(device), char_ctx.to(device), char_ques.to(device), label.to(device)

        with torch.no_grad():
            s_idx, e_idx = label[:, 0], label[:, 0]
            preds = model(context, question, char_ctx, char_ques)

            p1, p2 = preds
            loss = F.cross_entropy(p1, s_idx) + F.cross_entropy(p2, e_idx)
            valid_loss += loss.item()

            batch_size, c_len = p1.size()
            ls = nn.LogSoftmax(dim=1)
            mask = (torch.ones(c_len, c_len) * float('-inf')).to(device).tril(-1).unsqueeze(0).expand(batch_size, -1, -1)
            score = (ls(p1).unsqueeze(2) + ls(p2).unsqueeze(1)) + mask
            score, s_idx = score.max(dim=1)
            score, e_idx = score.max(dim=1)
            s_idx = torch.gather(s_idx, 1, e_idx.view(-1, 1)).squeeze()
            
           
            for i in range(batch_size):
                id = ids[i]
                pred = context[i][s_idx[i]:e_idx[i]+1]
                pred = ' '.join([idx2word[idx.item()] for idx in pred])
                predictions[id] = pred   

    
    em, f1 = evaluate(predictions)
    print("done")
    return valid_loss/len(valid_dataset), em, f1


In [47]:
def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
        
    return max(scores_for_ground_truths)


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [50]:
def evaluate(predictions):

    with open("../data/squad_dev.json", "r", encoding="utf-8") as f:
        dataset = json.load(f)

    dataset = dataset["data"]
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraphs in article["paragraphs"]:
            for qa in paragraphs["qas"]:
                total+=1
                if qa["id"] not in predictions:
                    continue
                
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)
                
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    
    return exact_match, f1

In [51]:
train_losses = []
valid_losses = []
ems = []
f1s = []
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    start_time = time.time()
    
    train_loss = train(model, train_dataset)
    valid_loss, em, f1 = valid(model, valid_dataset)
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss,
            'em':em,
            'f1':f1,
            }, 'bidaf_run4_{}.pth'.format(epoch))
    
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    ems.append(em)
    f1s.append(f1)

    print(f"Epoch train loss : {train_loss}| Time: {epoch_mins}m {epoch_secs}s")
    print(f"Epoch valid loss: {valid_loss}")
    print(f"Epoch EM: {em}")
    print(f"Epoch F1: {f1}")
    print("====================================================================================")

Epoch 1
training


  0%|          | 0/7 [00:00<?, ?it/s]

Starting batch: 0


100%|██████████| 7/7 [00:29<00:00,  4.25s/it]


Validating....
Starting batch 0
done
Epoch train loss : 9.745165961129326| Time: 0m 35s
Epoch valid loss: 8.648008584976196
Epoch EM: 0.0
Epoch F1: 0.003784295175023652
Epoch 2
training


  0%|          | 0/7 [00:00<?, ?it/s]

Starting batch: 0


 57%|█████▋    | 4/7 [00:21<00:15,  5.30s/it]


KeyboardInterrupt: ignored